In [1]:
import os
import copy
import shutil
import pandas as pd

In [2]:
# set path
# first copy original files into fmt_path
raw_path = "D:\\EGI_DATA\\EGI_RAW"
fmt_path = "D:\\EGI_DATA\\EGI_FMT"
edf_path = "D:\\EGI_DATA\\EGI_EDF"
bids_root_path = "D:\\EGI_DATA\\EGI_BIDS"
beh_path = "beh"
rec_path = "rec"
eeg_path = "eeg"
# set task name
wordtask = "wordchoice"
imagetask = "imagechoice"
foodtask = "foodchoice"
wmtask = "wordimagematch"
rest = "rest"

# 1. format raw data

In [5]:
if os.path.exists(fmt_path):
    shutil.rmtree(fmt_path)
else:
    os.mkdir(fmt_path)
task_set = set(["foodEnd", "foodChoice", "foodHealthy", "imageChoice", "wordChoice", "wordImageMatch", "foodTaste"])
task_dict = {"F": "foodchoice", "FC": "foodchoice", "I": "imagechoice", "IC": "imagechoice", "W": "wordchoice", "WC": "wordchoice", "R": "rest", "RS": "rest", "WM": "wordimagematch", "WM": "wordimagematch"}
total = 0
for sub in os.listdir(raw_path):
    # under "sub-xxx" directory
    if not os.path.exists(os.path.join(fmt_path, sub, eeg_path)):
        os.makedirs(os.path.join(fmt_path, sub, eeg_path))
    if not os.path.exists(os.path.join(fmt_path, sub, beh_path)):
        os.makedirs(os.path.join(fmt_path, sub, beh_path))
    for directory in os.listdir(os.path.join(raw_path, sub)):
        # eeg data
        if "EEG DATA" in directory or "EEG_DATA" in directory:
            # sub-001 to sub-015
            cur_eeg_path = os.path.join(raw_path, sub, directory)
            if sub <= "sub-015":
                cur_eeg_path = os.path.join(cur_eeg_path, "ori_mff")
            for mff in os.listdir(cur_eeg_path):
                if "fil.mff" in mff or "fil.edf" in mff:
                    continue
                for key, value in task_dict.items():
                    if key in mff[:-4].upper().replace('-', ' ').replace('_', ' ').split():
                        print(os.path.join(cur_eeg_path, mff))
                        shutil.copytree(os.path.join(cur_eeg_path, mff), os.path.join(fmt_path, sub, eeg_path, "{}_task-{}_eeg.mff".format(sub, value)))
                        break
        # behavioral data
        elif "Beha" in directory:
            for csv in os.listdir(os.path.join(raw_path, sub, directory)):
                cross = set(csv.split("_")) & task_set
                if len(cross) != 0:
                    total += 1
                    # print(os.path.join(fmt_path, sub, beh_path, csv))
                    shutil.copy(os.path.join(raw_path, sub, directory, csv), os.path.join(fmt_path, sub, beh_path, "{}_task-{}_beh.csv".format(sub, list(cross)[0]).lower()))

## delete behavioral data and wordimagematch eeg data

In [9]:
for sub in os.listdir(fmt_path):
    shutil.rmtree(os.path.join(fmt_path, sub, beh_path), ignore_errors=True)
    shutil.rmtree(os.path.join(fmt_path, sub, eeg_path, "{}_task-{}_eeg.mff".format(sub, wmtask)), ignore_errors=True)

# 2. convert .mff file to .edf file

## convert mff to edf

In [16]:
from mff2edf import write_mne_edf

In [17]:
# if os.path.exists(edf_path):
#     shutil.rmtree(edf_path)
# else:
#     os.mkdir(edf_path)

In [19]:
if os.path.exists(edf_path):
    shutil.rmtree(edf_path)
else:
    os.mkdir(edf_path)
# place edf files in edf_path
for sub in os.listdir(fmt_path):
    # create sub/eeg path if not exist
    if not os.path.exists(os.path.join(edf_path, sub)):
        os.makedirs(os.path.join(edf_path, sub, eeg_path))
    for mff in os.listdir(os.path.join(fmt_path, sub, eeg_path)):
        edf_file_path = os.path.join(edf_path, sub, eeg_path, "{}edf".format(mff[:-3]))
        if not os.path.exists(edf_file_path):
            write_mne_edf(os.path.join(fmt_path, sub, eeg_path, mff), edf_file_path, overwrite=True)

Reading EGI MFF Header from D:\EGI_DATA\EGI_FMT\sub-001\eeg\sub-001_task-foodchoice_eeg.mff...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {} ...
Reading 0 ... 1716132  =      0.000 ...  1716.132 secs...
saving to D:\EGI_DATA\EGI_EDF\sub-001\eeg\sub-001_task-foodchoice_eeg.edf, filetype 1
961 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 14

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\EGI_DATA\\EGI_FMT\\sub-012\\eeg\\sub-012_task-foodchoice_eeg.mff\\info1.xml'

# 3. BIDS

## convert to bids

In [ ]:
import mne
from mne_bids import write_raw_bids, BIDSPath, print_dir_tree

In [ ]:
if os.path.exists(bids_root_path):
    shutil.rmtree(bids_root_path)
else:
    os.makedirs(bids_root_path)

In [ ]:
# TODO
for sub in os.listdir(edf_path):
    # create sub/eeg path if not exist
    # if not os.path.exists(os.path.join(edf_path, sub)):
    #     os.makedirs(os.path.join(edf_path, sub, eeg_path))
    for edf_file in os.listdir(os.path.join(edf_path, sub, eeg_path)):
        subject_id = edf_file.split("_")[0].split("-")[1]
        task = edf_file.split("_")[1].split("-")[1]
        bids_path = BIDSPath(subject=subject_id, task=task, root=bids_root_path)
        raw = mne.io.read_raw_edf(os.path.join(edf_path, sub, eeg_path, edf_file), preload=False)
        raw.info["line_freq"] = 50
        write_raw_bids(raw, bids_path, overwrite=True)